# Neutryx XVA Calculation Demo

This notebook demonstrates XVA (X-Value Adjustments) calculations:
- CVA (Credit Valuation Adjustment)
- DVA (Debt Valuation Adjustment)
- FVA (Funding Valuation Adjustment)
- ColVA (Collateral Valuation Adjustment)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

## 1. Market Data Setup

In [ ]:
# Load credit spread data
credit_data = pd.read_csv('../data/input/market_data/credit_spreads/spreads.csv')
print("Credit Spreads Sample:")
print(credit_data[credit_data['reference_entity'] == 'JPMORGAN'].head())

In [ ]:
# Build survival probability curves
def survival_probability(spread_bps, recovery, T):
    """Calculate survival probability from CDS spread."""
    hazard_rate = (spread_bps / 10000) / (1 - recovery)
    return np.exp(-hazard_rate * T)

# Example: JPMorgan survival curve
jpm_data = credit_data[credit_data['reference_entity'] == 'JPMORGAN']
tenors = jpm_data['tenor_years'].values
spreads = jpm_data['spread_bps'].values
recovery = jpm_data['recovery_rate'].values[0]

survival_probs = [survival_probability(s, recovery, t) for s, t in zip(spreads, tenors)]

plt.figure(figsize=(10, 6))
plt.plot(tenors, survival_probs, 'bo-', label='JPMorgan')

# Add another counterparty for comparison
tesla_data = credit_data[credit_data['reference_entity'] == 'TESLA']
if len(tesla_data) > 0:
    tesla_survival = [survival_probability(s, tesla_data['recovery_rate'].values[0], t) 
                      for s, t in zip(tesla_data['spread_bps'].values, tesla_data['tenor_years'].values)]
    plt.plot(tesla_data['tenor_years'].values, tesla_survival, 'ro-', label='Tesla')

plt.xlabel('Time (Years)')
plt.ylabel('Survival Probability')
plt.title('Counterparty Survival Probability Curves')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. CVA Calculation

CVA represents the expected loss from counterparty default.

In [ ]:
def calculate_cva(ee_profile, time_grid, survival_curve, recovery_rate, discount_curve):
    """
    Calculate CVA using numerical integration.
    
    CVA = (1-R) * integral[0,T] EE(t) * dPD(t) * D(t)
    """
    lgd = 1 - recovery_rate
    cva = 0
    
    for i in range(1, len(time_grid)):
        t = time_grid[i]
        t_prev = time_grid[i-1]
        dt = t - t_prev
        
        # Interpolate values
        ee = ee_profile[i]
        sp_prev = survival_curve(t_prev)
        sp_curr = survival_curve(t)
        pd = sp_prev - sp_curr  # Default probability in period
        df = discount_curve(t)
        
        cva += lgd * ee * pd * df
    
    return cva

# Generate EE profile (from previous notebook)
time_grid = np.linspace(0, 10, 41)
base_exposure = 10_000_000  # $10M base exposure
ee_profile = base_exposure * np.exp(-0.05 * time_grid) * (1 + 0.3 * np.sin(time_grid))
ee_profile = np.maximum(ee_profile, 0)

# Create interpolation functions
survival_interp = CubicSpline(tenors, survival_probs, extrapolate=True)
discount_interp = lambda t: np.exp(-0.04 * t)  # Simple discount curve

# Calculate CVA
cva = calculate_cva(ee_profile, time_grid, survival_interp, recovery, discount_interp)
print(f"CVA: ${cva:,.0f}")
print(f"CVA as % of notional: {cva/base_exposure*100:.2f}%")

## 3. DVA Calculation

DVA represents the benefit from our own default (symmetric to CVA).

In [ ]:
def calculate_dva(nee_profile, time_grid, own_survival_curve, own_recovery, discount_curve):
    """
    Calculate DVA using numerical integration.
    
    DVA = (1-R_own) * integral[0,T] NEE(t) * dPD_own(t) * D(t)
    """
    lgd = 1 - own_recovery
    dva = 0
    
    for i in range(1, len(time_grid)):
        t = time_grid[i]
        t_prev = time_grid[i-1]
        
        nee = nee_profile[i]
        sp_prev = own_survival_curve(t_prev)
        sp_curr = own_survival_curve(t)
        pd = sp_prev - sp_curr
        df = discount_curve(t)
        
        dva += lgd * nee * pd * df
    
    return dva

# Our own survival curve (assume investment grade)
own_spread_bps = 50  # 50bp spread
own_recovery = 0.40
own_hazard = (own_spread_bps / 10000) / (1 - own_recovery)
own_survival = lambda t: np.exp(-own_hazard * t)

# NEE profile (negative expected exposure)
nee_profile = base_exposure * 0.3 * np.exp(-0.03 * time_grid) * (1 + 0.2 * np.cos(time_grid))
nee_profile = np.maximum(nee_profile, 0)

# Calculate DVA
dva = calculate_dva(nee_profile, time_grid, own_survival, own_recovery, discount_interp)
print(f"DVA: ${dva:,.0f}")
print(f"DVA as % of notional: {dva/base_exposure*100:.2f}%")

## 4. FVA Calculation

FVA represents the cost of funding uncollateralised exposure.

In [ ]:
def calculate_fva(ee_profile, nee_profile, time_grid, funding_spread, discount_curve):
    """
    Calculate FVA.
    
    FVA = integral[0,T] (EE(t) * funding_spread_borrow + NEE(t) * funding_spread_lend) * D(t) * dt
    """
    fva = 0
    
    for i in range(1, len(time_grid)):
        t = time_grid[i]
        dt = t - time_grid[i-1]
        
        ee = ee_profile[i]
        nee = nee_profile[i]
        df = discount_curve(t)
        
        # Funding cost on positive exposure
        fca = ee * funding_spread * df * dt
        # Funding benefit on negative exposure (assume lower spread)
        fba = nee * (funding_spread * 0.5) * df * dt
        
        fva += fca - fba
    
    return fva

# Funding spread
funding_spread = 0.008  # 80bp funding spread

# Calculate FVA
fva = calculate_fva(ee_profile, nee_profile, time_grid, funding_spread, discount_interp)
print(f"FVA: ${fva:,.0f}")
print(f"FVA as % of notional: {fva/base_exposure*100:.2f}%")

## 5. XVA Summary

In [ ]:
# Total XVA
total_xva = cva - dva + fva

# Summary table
xva_summary = pd.DataFrame({
    'Metric': ['CVA', 'DVA', 'FVA', 'Total XVA'],
    'Value ($)': [cva, -dva, fva, total_xva],
    '% of Notional': [cva/base_exposure*100, -dva/base_exposure*100, 
                     fva/base_exposure*100, total_xva/base_exposure*100]
})

print("XVA Summary:")
print(xva_summary.to_string(index=False))

# Visualisation
plt.figure(figsize=(10, 6))
xvas = ['CVA', 'DVA', 'FVA', 'Net XVA']
values = [cva/1e6, -dva/1e6, fva/1e6, total_xva/1e6]
colors = ['red', 'green', 'orange', 'blue']
plt.bar(xvas, values, color=colors, alpha=0.7)
plt.ylabel('Value ($ millions)')
plt.title('XVA Breakdown')
plt.axhline(y=0, color='black', linewidth=0.5)
plt.grid(True, axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(values):
    plt.text(i, v + 0.01, f'${v:.2f}M', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## 6. XVA Sensitivity Analysis

In [ ]:
# CVA sensitivity to counterparty credit spread
spread_range = np.linspace(20, 200, 19)  # 20bp to 200bp
cva_sensitivity = []

for spread in spread_range:
    hazard = (spread / 10000) / (1 - recovery)
    surv = lambda t, h=hazard: np.exp(-h * t)
    cva_val = calculate_cva(ee_profile, time_grid, surv, recovery, discount_interp)
    cva_sensitivity.append(cva_val)

plt.figure(figsize=(10, 6))
plt.plot(spread_range, [c/1e6 for c in cva_sensitivity], 'b-', linewidth=2)
plt.xlabel('Counterparty Credit Spread (bp)')
plt.ylabel('CVA ($ millions)')
plt.title('CVA Sensitivity to Counterparty Credit Spread')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
1. CVA calculation from exposure profiles and credit curves
2. DVA calculation considering our own default risk
3. FVA for funding costs on uncollateralised derivatives
4. XVA aggregation and sensitivity analysis

Neutryx's `pricer_risk::xva` module provides high-performance Monte Carlo simulation for XVA calculations.